In [8]:
from osgeo import gdal, ogr, osr
import numpy as np
import geopandas as gpd
from shapely.geometry import box

# --- Paramètres ---
input_vector_path = "../../data/project/FORMATION_VEGETALE.shp"  # Chemin vers la BD Forêt
ref_raster_path = "../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif"  # Raster Sentinel-2 de référence
output_rasterized_path = "../results/data/sample/rasterized_roi.tif"
output_folder = "../results/data/sample"
output_file = os.path.join(output_folder, "Sample_BD_foret_T31TCJ.shp")
reprojected_raster_path = "../results/data/sample/SENTINEL2B_reprojected_2154.tif"

# Définir le système de coordonnées cible (EPSG:2154)
target_crs = "EPSG:2154"

gdal.Warp(
    reprojected_raster_path,       # Chemin de sortie
    ref_raster_path,               # Chemin d'entrée
    dstSRS=target_crs,             # CRS cible
    resampleAlg=gdal.GRA_Bilinear, # Méthode de rééchantillonnage (bilinéaire)
    format="GTiff"                 # Format de sortie
)

# Liste des classes valides
codes_tfv_valides = [
    "Forêt fermée d’un autre feuillu pur",
    "Forêt fermée de châtaignier pur",
    "Forêt fermée de hêtre pur",
    "Forêt fermée de chênes décidus purs",
    "Forêt fermée de robinier pur",
    "Peuplerai",
    "Forêt fermée à mélange de feuillus",
    "Forêt fermée de feuillus purs en îlots",
    "Forêt fermée d’un autre conifère pur autre que pin",
    "Forêt fermée de mélèze pur",
    "Forêt fermée de sapin ou épicéa",
    "Forêt fermée à mélange d’autres conifères",
    "Forêt fermée d’un autre pin pur",
    "Forêt fermée de pin sylvestre pur",
    "Forêt fermée à mélange de pins purs",
    "Forêt fermée de douglas pur",
    "Forêt fermée de pin laricio ou pin noir pur",
    "Forêt fermée de pin maritime pur",
    "Forêt fermée à mélange de conifères",
    "Forêt fermée de conifères en îlots",
    "Forêt fermée à mélange de conifères prépondérants et feuillus",
    "Forêt fermée à mélange de feuillus prépondérants et conifères"
]

# Création du dossier de sortie si inexistant
os.makedirs(output_folder, exist_ok=True)

# --- Étape 1 : Filtrer les polygones par les classes TFV ---
bd_foret = gpd.read_file(input_vector_path)
bd_foret_filtered = bd_foret[bd_foret["TFV"].isin(codes_tfv_valides)]
filtered_vector_path = os.path.join(output_folder, "filtered_tfv.shp")
bd_foret_filtered.to_file(filtered_vector_path)

# --- Étape 2 : Rasterisation des polygones avec GDAL ---
# Ouvrir l'image raster de référence pour obtenir les paramètres géospatiaux
ref_ds = gdal.Open(reprojected_raster_path)
geo_transform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
x_res = ref_ds.RasterXSize
y_res = ref_ds.RasterYSize

# Créer le raster de sortie
target_ds = gdal.GetDriverByName('GTiff').Create(output_rasterized_path, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform(geo_transform)
target_ds.SetProjection(projection)
target_ds.GetRasterBand(1).SetNoDataValue(0)  # Valeur NoData pour le raster

# Rasteriser le vecteur filtré
vector_ds = ogr.Open(filtered_vector_path)
layer = vector_ds.GetLayer()
gdal.RasterizeLayer(target_ds, [1], layer, options=["ATTRIBUTE=TFV"])  # Rasterisation en utilisant l'attribut "TFV"

target_ds = None
vector_ds = None
print(f"Rasterisation terminée. Raster sauvegardé : {output_rasterized_path}")

# --- Étape 3 : Extraction des pixels correspondant aux polygones ---
print("Étape 3: Extraction des pixels avec GDAL...")

# Ouvrir le raster de référence
raster_ds = gdal.Open(reprojected_raster_path)
raster_band = raster_ds.GetRasterBand(1)
raster_data = raster_band.ReadAsArray()

# Ouvrir le masque rasterisé
mask_ds = gdal.Open(output_rasterized_path)
mask_band = mask_ds.GetRasterBand(1)
mask_data = mask_band.ReadAsArray()

# Extraire les pixels dont la valeur dans le masque est non nulle
valid_mask = mask_data > 0
extracted_pixels = raster_data[valid_mask]
y_indices, x_indices = np.where(valid_mask)

# Créer un GeoDataFrame contenant les coordonnées des pixels extraits
print(f"Nombre de pixels extraits : {len(extracted_pixels)}")
geo_transform = raster_ds.GetGeoTransform()
geometry = [box(
    geo_transform[0] + x * geo_transform[1],
    geo_transform[3] + y * geo_transform[5],
    geo_transform[0] + (x + 1) * geo_transform[1],
    geo_transform[3] + (y + 1) * geo_transform[5],
) for x, y in zip(x_indices, y_indices)]

# Sauvegarder en Shapefile
sample_gdf = gpd.GeoDataFrame({"Code": extracted_pixels, "Nom": "Classified Pixel"}, geometry=geometry)
sample_gdf.to_file(output_file, driver="ESRI Shapefile")

print(f"Fichier final créé : {output_file}")


Rasterisation terminée. Raster sauvegardé : ../results/data/sample/rasterized_roi.tif
Étape 3: Extraction des pixels avec GDAL...
Nombre de pixels extraits : 0
Fichier final créé : ../results/data/sample/Sample_BD_foret_T31TCJ.shp


/opt/conda/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
